In [1]:
# set up
import os
import time
import numpy as np
import pandas as pd
import functools as func
import functions_v3 as f
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
home_path = "C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen"
os.chdir(home_path)

In [ ]:
os.chdir(home_path)
os.chdir('use_case_1/')
use_case_home_path = os.getcwd()
df_mm = f.share_renewable_get_mm_quantity(use_case_home_path)
df_mm

In [ ]:
df_mm['mm_grey_energy [%]'].mean()

#### KPI 4: CO2 emissions
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [ ]:
results_path = 'C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen\\results'
os.chdir(home_path)
all_filepaths = []
for root, dir, file in os.walk(top=results_path, topdown=True):
    all_filepaths += [os.path.join(root, f) for f in file if 'kpi_3_4' in f]

In [ ]:
results_path = 'C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen\\results'
os.chdir(home_path)

all_filepaths = []
for root, dir, file in os.walk(top=results_path, topdown=True):
    all_filepaths += [os.path.join(root, f) for f in file if 'kpi_3_4' in f]
dfs = []
for i in range(7):
    df_temp = pd.read_excel(all_filepaths[i], index_col=0)
    dfs.append(df_temp)

In [ ]:
pd.concat([dfs[i][['region-5-ec5_grey_energy [kWh]']] for i in range(len(dfs))], axis=1)

In [ ]:
os.chdir(home_path)
os.chdir('Use_Case_2')
use_case_home_path = os.getcwd()
# calculate net renewable share for mm over all entities that trade with it
df_mm = f.share_renewable_get_mm_quantity(use_case_home_path)
# for use case, extract all (non-)relevant filepaths and construct empty df_out with all time steps
nonempty_files_paths, empty_files_paths, nr_months, l, df_out = f.prepare_filepaths(use_case_home_path)
# combine each entity's monthly data frames and hand them over to shares_renewable_helper for calculation

In [ ]:
i = 0
entity_monthly_dfs = []
entity_monthly_filepaths = nonempty_files_paths[i::int(l / nr_months)]
entity_monthly_filepaths

In [ ]:
for j in entity_monthly_filepaths:
    df_temp = pd.read_csv(j).drop(['creation_time', 'matching_requirements', 'rate [ct./kWh]'], axis=1)
    df_temp.seller = [i.lower().replace('_', '-') for i in df_temp.seller]
    df_temp.buyer = [i.lower().replace('_', '-') for i in df_temp.buyer]
    entity_monthly_dfs.append(df_temp)
entity_name = entity_monthly_filepaths[0].split('\\')[-1].split('-trades.csv')[0]
entity_df = pd.concat(entity_monthly_dfs).sort_values(by='slot').reset_index(drop=True)
entity_df.set_index(['slot'], inplace=True)
# hand over entity_df to helper function which inserts entity's result as new column into df_out
df_out = f.share_renewable_helper(df_out, df_mm, entity_df, entity_name, co2=True)

In [ ]:
df_out

In [ ]:
p = 'grid'
f'{p}_grey_energy [kWh]' in df_out.columns.tolist()

In [3]:
# define function for execution convenience
def calculate_kpi_3_4(use_case_string):
    os.chdir(home_path)
    os.chdir(use_case_string)
    use_case_home_path = os.getcwd()
    print(f'-- use case {f.get_use_case_nr(use_case_home_path)} --')
    start = time.time()
    df = f.co2_emissions(use_case_home_path)
    print(f'time: {round(time.time() - start, 2)} seconds')
    return df

In [ ]:
df0 = calculate_kpi_3_4('Base_Case')
df0

In [ ]:
df0.to_excel('kpi_3_4_use_case_0.xlsx')

In [ ]:
df1 = calculate_kpi_3_4('use_case_1/')
df1

In [ ]:
df1.to_excel('kpi_3_4_use_case_1.xlsx')

In [4]:
df2 = calculate_kpi_3_4('Use_Case_2/')
df2

-- use case 2 --
time: 123.38 seconds


,grid_grey_energy [kWh],grid_grey_energy [%],region-5_grey_energy [kWh],region-5_grey_energy [%],region-4_grey_energy [kWh],region-4_grey_energy [%],region-2_grey_energy [kWh],region-2_grey_energy [%],region-1_grey_energy [kWh],region-1_grey_energy [%],...,region-6-ec4-house-7_id-7-ev-non-commuter_grey_energy [kWh],region-6-ec4-house-7_id-7-ev-non-commuter_grey_energy [%],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [kWh],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [%],region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [%],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [%],region-6-industry-electric-load_grey_energy [kWh],region-6-industry-electric-load_grey_energy [%]
2030-01-01T00:00,770.863572,0.738527,97.315666,0.738527,99.961676,0.738527,202.812608,0.738527,-0.000000,0.000000,...,0.001807,0.451061,1.411557,0.451061,0.495445,0.451061,0.495445,0.451061,29.190511,0.451061
2030-01-01T01:00,808.200861,0.877856,58.027228,0.629050,120.664573,0.877856,242.205502,0.877856,75.072711,0.877856,...,0.007593,0.438434,1.368097,0.438434,0.455387,0.438434,0.455387,0.438434,27.522390,0.438434
2030-01-01T02:00,767.327944,0.875751,64.575681,0.680355,117.273249,0.875751,234.791439,0.875751,72.574884,0.875751,...,0.004153,0.346454,1.078174,0.346454,0.365763,0.346454,0.365763,0.346454,23.903718,0.374204
2030-01-01T03:00,806.917892,0.850986,112.339667,0.850986,115.738706,0.850986,221.307824,0.850986,69.561542,0.850986,...,0.003685,0.670362,2.081051,0.670362,1.295138,0.670362,1.295138,0.670362,43.397643,0.670362
2030-01-01T04:00,920.101311,0.819207,115.737565,0.802489,105.977018,0.819207,153.414860,0.658573,72.594214,0.819207,...,0.003257,0.730790,2.263678,0.730790,1.359269,0.730790,1.359269,0.730790,47.623602,0.730790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-10-07T19:00,181.346512,0.186586,15.120877,0.186586,-0.000000,0.000000,27.794809,0.186586,4.822307,0.186586,...,0.004319,0.043293,0.122996,0.043293,0.047103,0.043293,0.047103,0.043293,2.894559,0.047730
2030-10-07T20:00,161.209832,0.325583,-0.000000,0.000000,20.611906,0.325583,29.373716,0.293393,0.000000,0.000000,...,0.002951,0.035381,0.100172,0.035381,0.042598,0.035381,0.042598,0.035381,2.273731,0.037894
2030-10-07T21:00,184.617216,0.192502,18.204144,0.192502,23.025502,0.192502,-0.000000,0.000000,6.782434,0.192502,...,0.002081,0.029925,0.084437,0.029925,0.053506,0.029925,0.053506,0.029925,1.820168,0.030150
2030-10-07T22:00,203.386686,0.233204,28.036288,0.233204,-0.000000,0.000000,34.487333,0.233204,3.100407,0.233204,...,0.001135,0.018224,0.051247,0.018224,0.031501,0.018224,0.031501,0.018224,1.350065,0.022663


In [5]:
df2['grid_grey_energy [%]'].mean()

0.4135625705219909

In [ ]:
df2.to_excel('kpi_3_4_use_case_2.xlsx')

In [ ]:
df3 = calculate_kpi_3_4('use_case_3/')
df3

In [ ]:
df3.to_excel('kpi_3_4_use_case_3.xlsx')

In [ ]:
df4 = calculate_kpi_3_4('use_case_4/')
df4

In [ ]:
df4.to_excel('kpi_3_4_use_case_4.xlsx')

In [ ]:
# use case 5
df5 = calculate_kpi_3_4('use_case_5')
df5

In [ ]:
df5.to_excel('kpi_3_4_use_case_5.xlsx')

In [ ]:
# use case 6
df6 = calculate_kpi_3_4('use_case_6')
df6

In [ ]:
df6.to_excel('kpi_3_4_use_case_6.xlsx')